In [ ]:
from sklearn.metrics import classification_report
import pandas as pd, warnings, seaborn as sns, matplotlib.pyplot as plt, plotly.express as px, numpy as np
from sklearn import preprocessing, metrics
from pycaret.classification import *
from imblearn.combine import SMOTEENN, SMOTETomek
from imblearn.over_sampling import (RandomOverSampler, SMOTE, BorderlineSMOTE, SVMSMOTE, ADASYN)
from imblearn.under_sampling import(
    RandomUnderSampler, NearMiss, ClusterCentroids, TomekLinks, EditedNearestNeighbours,CondensedNearestNeighbour)
datasett = pd.read_csv('D:/dataset.csv', index_col = 'patient_id', parse_dates = True)
datasett.age = datasett.age.apply(lambda x : x if x > 0 else 0)
datasett.patient_created_date=pd.to_datetime(datasett.patient_created_date)
target_date = pd.to_datetime('2023-03-01')
datasett['account_length_days'] = (target_date - datasett['patient_created_date']).dt.days
datasett.drop(columns = ['clinic_id','patient_created_date','plan_policy_type_1','plan_policy_type',
                         'gender','total_fee','age','claim_fee'], inplace = True)
dataset_1 = datasett[datasett['Attachment']==1]
dataset_0 = datasett[datasett['Attachment']==0]
dataset_0 = dataset_0.sample(n=159733)
dataset = pd.concat([dataset_0,dataset_1])
dataset = dataset.dropna()
target_date = pd.to_datetime('2023-03-01')
s = setup(data = dataset, target = 'Attachment', train_size=0.8, normalize = True, categorical_features = ['state', 
                                   'zip_code', 'cdt_code','insurance_plan_name','fee_schedule_name', 'insurance_type', 
                                  'plan_year_type'])
rf = create_model('et')
final_et = finalize_model(rf)
save_model(final_et,'RF2')
saved_final_rf = load_model('RF2')
data_unseen_clinics = pd.read_csv('D:/dt.csv', index_col ='patient_id', parse_dates = True)
target_date = pd.to_datetime('2023-02-24')
data_unseen_clinics['patient_created_date'] = pd.to_datetime(data_unseen_clinics['patient_created_date'])
data_unseen_clinics['account_length_days'] = (target_date - data_unseen_clinics['patient_created_date']).dt.days
data_unseen_clinics = data_unseen_clinics.drop(columns = ['clinic_id','total_fee','patient_created_date'])
unseen_predictions_clinics_2 = predict_model(saved_final_rf, data=data_unseen_clinics)
data_unseen_clinics = pd.read_csv('D:/dt.csv', index_col ='patient_id', parse_dates = True)
print(classification_report(unseen_predictions_clinics_2['Attachment'], unseen_predictions_clinics_2['prediction_label']))

In [2]:
app = create_app(saved_final_rf) 

Running on local URL:  http://127.0.0.1:7868

To create a public link, set `share=True` in `launch()`.
